# 🤖 Amazon Bedrock AgentCore Browser Tool

Welcome to this hands-on lab where you'll learn to integrate Amazon Bedrock AgentCore's browser tool into your agents. This tool enables agents to interact with web pages through automated browser sessions powered by Amazon NovaAct.

![](../media/browser_tool_arch.png)

## Learning Objectives

- Integrate Amazon Bedrock AgentCore's browser tool with agents
- Understand asynchronous tool execution patterns
- Build agents that can search and interact with websites like Amazon.com
- Handle browser automation results in conversational flows


In [ ]:
# Import dependencies. 

import boto3
import json

# Initialize the Bedrock client
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

print("✅ Setup complete!")

In the [Module 3 Lab 3 (Agent Tools)](../../module3/notebooks/3_agent_tools.ipynb), we created an Agent implementation with features like memory and tool calling. We'll now extend this concept by integrating Amazon Bedrock AgentCore's browser tool, which enables agents to interact with web pages through automated browser sessions.

## Reuse what we wrote in the previous lab. 

In [ ]:
from agentic_platform.core.models.prompt_models import BasePrompt
from agentic_platform.core.models.memory_models import Message, SessionContext, ToolResult, ToolCall
from agentic_platform.core.models.llm_models import LLMResponse, LLMResponse
from agentic_platform.core.converter.llm_request_converters import ConverseRequestConverter
from agentic_platform.core.converter.llm_response_converters import ConverseResponseConverter
from typing import Dict, Any, Optional, List, Type, Callable

# Helper to construct request
def construct_request(user_message: str, conversation_id: str=None) -> AgenticRequest:
    return AgenticRequest.from_text(
        text=user_message, 
        **{'session_id': conversation_id}
    )

# Helper function to call Bedrock. Passing around JSON is messy and error prone.
def call_bedrock(request: LLMRequest) -> LLMResponse:
    kwargs: Dict[str, Any] = ConverseRequestConverter.convert_llm_request(request)
    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return ConverseResponseConverter.to_llm_response(converse_response)

class MemoryClient:
    """Manages conversations"""
    def __init__(self):
        self.conversations: Dict[str, SessionContext] = {}

    def upsert_conversation(self, conversation: SessionContext, conversation_id: str=None) -> bool:
        if conversation_id:
            self.conversations[conversation_id] = conversation
        else:
            self.conversations[conversation.session_id] = conversation

    def get_or_create_conversation(self, conversation_id: str=None) -> SessionContext:
        return self.conversations.get(conversation_id, SessionContext()) if conversation_id else SessionContext()
    
memory_client: MemoryClient = MemoryClient()

In [ ]:
from pydantic import BaseModel

# Import our agent request and response types.
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse
from agentic_platform.core.models.memory_models import TextContent
from agentic_platform.core.models.tool_models import ToolSpec
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse


class ToolCallingAgent:
    # This is new, we're adding tools in the constructor to bind them to the agent.
    # Don't get too attached to this idea, it'll change as we get into MCP.
    def __init__(self, tools: List[ToolSpec], prompt: BasePrompt):
        self.tools: List[ToolSpec] = tools
        self.conversation: SessionContext = SessionContext()
        self.prompt: BasePrompt = prompt

    def call_llm(self) -> LLMResponse:
        # Create LLM request
        request: LLMRequest = LLMRequest(
            system_prompt=self.prompt.system_prompt,
            messages=self.conversation.get_messages(),
            model_id=self.prompt.model_id,
            hyperparams=self.prompt.hyperparams,
            tools=self.tools
        )

        # Call the LLM.
        response: LLMResponse = call_bedrock(request)
        # Append the llms response to the conversation.
        self.conversation.add_message(Message(
            role="assistant",
            text=response.text,
            tool_calls=response.tool_calls
        ))
        # Return the response.
        return response
    
    def execute_tools(self, llm_response: LLMResponse, session_id:str) -> List[ToolResult]:
        """Call tools and return the results."""
        # It's possible that the model will call multiple tools.
        tool_results: List[ToolResult] = []
        # Iterate over the tool calls and call the tool.
        for tool_invocation in llm_response.tool_calls:
            # Get the tool spec for the tool call.
            tool: ToolSpec = next((t for t in self.tools if t.name == tool_invocation.name), None)
            # Call the tool.
            input_data: BaseModel = tool.model.model_validate(tool_invocation.arguments)
            # Dynamically inject session_id if the model has it
            if hasattr(input_data, "session_id"):
                setattr(input_data, "session_id", session_id)
            function_result: str = str(tool.function(input_data))
            tool_response: ToolResult = ToolResult(
                id=tool_invocation.id,
                content=[TextContent(text=function_result)],
                isError=False
            )

            print(f"Tool response: {tool_response}")

            # Add the tool result to the list.
            tool_results.append(tool_response)

        # Add the tool results to the conversation
        message: Message = Message(role="user", tool_results=tool_results)
        self.conversation.add_message(message)
        
        # Return the tool results even though we don't use it.
        return tool_results
    
    def invoke(self, request: AgenticRequest) -> AgenticResponse:
        # Get or create conversation
        self.conversation = memory_client.get_or_create_conversation(request.session_id)
        # Add user message to conversation
        self.conversation.add_message(request.message)

        # Keep calling LLM until we get a final response
        while True:
            # Call the LLM
            response: LLMResponse = self.call_llm()
            
            # If the model wants to use tools
            if response.stop_reason == "tool_use":
                # Execute the tools
                self.execute_tools(response,request.session_id)
                # Continue the loop to get final response
                continue
            
            # If we get here, it's a final response 
            break

        # Save updated conversation
        memory_client.upsert_conversation(self.conversation,request.session_id)

        # Return our own type.
        return AgenticResponse(
            message=self.conversation.messages[-1], # Just return the last message
            session_id=request.session_id if request.session_id else self.conversation.session_id
        )

## Example: Amazon.com Search Agent


In this example, we demonstrate how to integrate a tool that performs **product searches on Amazon.com** using the `browser_client` tool from the [Amamazon Bedrock AgentCore](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-tool.html) module.

This tool uses a **headless browser session** to navigate Amazon.com, perform a product search, and return the result asynchronously. The actual browser automation is powered by **Amazon NovaAct**, a framework for driving browser behavior with LLMs. Please request an API key [here](https://nova.amazon.com/act). 

Once the background browser task completes, the result is saved into memory and can be retrieved during a follow-up turn in the conversation.

This tool is ideal for use cases such as:
- Looking up the price of a product on Amazon
- Comparing different models or versions


In [ ]:
class AgentPrompt(BasePrompt):
    system_prompt: str = (
        "You are a helpful assistant. Some tools return results asynchronously (e.g., background_browser). "
        "When such tools are used, you should not fabricate final answers. Instead, inform the user to wait. "
        "If the result appears later in memory (e.g., a message starting with '📄 Browser result:'), you can use that to respond."
    )
    user_prompt: str = "{user_message}"


In [ ]:
from bedrock_agentcore.tools.browser_client import browser_session
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from nova_act import NovaAct

import os
import threading
import time
import random
import logging


def call_browser_tool(request: str,session_id: str):
    """Launches the browser session asynchronously and returns immediately."""
    print(f"Received browser tool request: {request}")

    thread = threading.Thread(
        target=_run_amazon_search_task,
        args=(request,session_id),
        daemon=True
    )
    thread.start()

    return {"messages": [{"role": "tool", "content": "The agent has started a background web search. This may take a moment."}]}


def _run_amazon_search_task(request: str, session_id: str):
    request += " (do a very quick and brief search, the faster you return search results the better. For example, no need to click into the product description if you see the price on the main search results)"

    try:
        with browser_session("us-east-1") as client:
            print("Browser session started... waiting for it to be ready.")
            time.sleep(5)

            ws_url, headers = client.generate_ws_headers()
            starting_url = "https://www.amazon.com"


            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=os.environ["NOVA_ACT_API_KEY"],
                starting_page=starting_url,
            ) as nova_act:
                result = nova_act.act(prompt=request, max_steps=20)

                assistant_msg = Message(
                    role="assistant",
                    content=[TextContent(text=f"Prompt:{str(result.metadata.prompt)} 📄 Browser result:\n{result.response}")]
                )
                conversation = memory_client.get_or_create_conversation(session_id)
                conversation.add_message(assistant_msg)
                memory_client.upsert_conversation(conversation,session_id)
                
    except Exception as e:
        print(f"Error during background browser task: {e}")


In [ ]:
from pydantic import BaseModel

class BrowserToolInput(BaseModel):
    request: str
    session_id: str

In [ ]:
from agentic_platform.core.models.tool_models import ToolSpec

browser_tool_spec = ToolSpec(
    name="background_browser",
    description="Starts an asynchronous background browser search. This tool does NOT return the actual result. The result will be written to memory later and should be referenced in follow-up turns.",
    model=BrowserToolInput,
    function=lambda args: call_browser_tool(args.request,args.session_id)
)

In [ ]:
memory_client = MemoryClient()
agent = ToolCallingAgent(
    tools=[browser_tool_spec],
    prompt=AgentPrompt()
)

When you invoke the agent, it will launch a background task to search for products. To see this in action:

1. Navigate to the AWS Console and go to the Amazon Bedrock section
![](../media/go_to_bedrock_agentcore_console.png)


2. In the Browser Tool tab, you'll see your agent's browser sessions
   ![](../media/browser_use_tab.png)

3. You can see the built-in browser sandbox that AgentCore provides
   ![](../media/aws_built_in_browser_sandbox.png)

4. Click on "View Live Session" to see the browser in action
   ![](../media/click_view_live_session.png)

5. Watch as the agent interacts with the browser to search for products
   ![](../media/watch_the_agent_interact_with_browser.png)

In [ ]:
os.environ['NOVA_ACT_API_KEY']='your-api-key'

session_id = "shopping-session-1"

req1 = construct_request("Find the price of an apple watch series 10",session_id)

response = agent.invoke(req1)
# Print the response
print(response.message.model_dump_json(indent=2))

In [ ]:
req2 = construct_request("Did you find the price of the apple watch I asked?",session_id)

response = agent.invoke(req2)
# Print the response
print(response.message.model_dump_json(indent=2))

## Summary

In this lab, you learned how to integrate Amazon Bedrock AgentCore's browser tool into your agents. Key takeaways:

- **Browser Automation**: Agents can interact with web pages through headless browser sessions powered by Amazon NovaAct
- **Asynchronous Execution**: Browser tools run in the background and save results to memory for later retrieval
- **Real-world Integration**: Demonstrated product search capabilities on Amazon.com
- **Conversational Flow**: Handled browser automation results within multi-turn conversations

This pattern enables agents to access real-time web content and perform complex web interactions beyond simple API calls.

## Additional Resources

- [Amazon Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/)
- [Browser Tool Reference](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-tool.html)
- [Amazon NovaAct Framework](https://nova.amazon.com/act)
- [Module 3 Lab 3: Agent Tools](../../module3/notebooks/3_agent_tools.ipynb)
- [Module 4 Lab 6: Model Context Protocol](6_mcp.ipynb)
